### Unzipping Gram matrices

In [ ]:
%%capture
!unzip ./data/gram_mats/dset0.zip -d ./dset0/
!unzip ./data/gram_mats/dset1.zip -d ./dset1/
!unzip ./data/gram_mats/dset2.zip -d ./dset2/

### Using KRRBags for each dataset

In [ ]:
import numpy as np
import multiprocessing as mp

from krr import KRRBag
from utils import eval_acc
from data import save_mat, load_mat, load_test, load_train

Loading gram matrices

In [ ]:
N_MODELS = 7

def load_gram_mats(dset):
  gram_train, gram_test = [], []
  for i in range(N_MODELS):
    train_f = "dset%d/dset%d_split_%d_Ktrain" % (dset, dset, i)
    test_f  = "dset%d/dset%d_split_%d_Ktest"  % (dset, dset, i)

    gram_train += [load_mat(train_f)]
    gram_test  += [load_mat(test_f)]

  return gram_train, gram_test

def reorder(l1, l2, p):
  l1_p, l2_p = [], []
  for i in p:
    l1_p += [l1[i]]
    l2_p += [l2[i]]

  return l1_p, l2_p

Dataset 0

In [ ]:
gram_train, gram_test = load_gram_mats(0)
gram_train, gram_test = reorder(gram_train, gram_test, [3,5,6,4,0,1,2])
_, _, Y0_train, _ = load_train(0, 1.)

In [ ]:
# Fitting
krr_bag = KRRBag(n_models=N_MODELS, ratio=0.95,
                 gram_train=gram_train, gram_test=gram_test,
                 l=0.11)
krr_bag.fit_K(Y0_train, verbose=True)

# Predicting
pred_0, preds = krr_bag.predict_K(verbose=True)

Dataset 1

In [ ]:
gram_train, gram_test = load_gram_mats(1)
gram_train, gram_test = reorder(gram_train, gram_test, [6,3,4,2,5,0,1])
_, _, Y1_train, _ = load_train(1, 1.)

In [ ]:
# Fitting
krr_bag = KRRBag(n_models=N_MODELS, ratio=0.95,
                 gram_train=gram_train, gram_test=gram_test,
                 l=0.0365)
krr_bag.fit_K(Y1_train, verbose=True)

# Predicting
pred_1, preds = krr_bag.predict_K(verbose=True)

Dataset 2

In [ ]:
gram_train, gram_test = load_gram_mats(2)
gram_train, gram_test = reorder(gram_train, gram_test, [0,1,2,3,4,5,6])
_, _, Y2_train, _ = load_train(2, 1.)

In [ ]:
# Fitting
krr_bag = KRRBag(n_models=N_MODELS, ratio=0.95,
                 gram_train=gram_train, gram_test=gram_test,
                 l=0.0256)
krr_bag.fit_K(Y2_train, verbose=True)

# Predicting
pred_2, preds = krr_bag.predict_K(verbose=True)

Building final submission

In [ ]:
import pandas as pd

all_preds = np.r_[pred_0, pred_1, pred_2]

df = pd.DataFrame(data=np.c_[np.arange(len(all_preds)), all_preds], columns=['Id','Bound'])
df.to_csv('Yte.csv', encoding='utf-8', index=False)